# Select Model

### For this notebook, the only supported method is Unsupervised method. The variable is only used to ensure code reuse and for ensuring file naming convention for later.

### Please do not change the "tqmodel" variable value in this notebook.

In [ ]:
tqmodel = "unsupervised"

# Select Language

### You will need to select one source language for Unsupervised QE method evaluation, at a time. 
### The source language can be specified with the variable "source_lang" defined below. 
### The valid values for this variable are:

* ru   ->   (Russian)
* ro   ->   (Romanian)
* si   ->   (Sinhala)
* et   ->   (Estonian)
* ne   ->   (Nepali)

In [ ]:
source_lang = "si" 

In [ ]:
import torch
import pandas as pd
import io, sys, os
from functools import reduce
import spacy
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import numpy as np
import random
import re
nlp = English()
tokenizer = nlp.tokenizer
from tqdm.notebook import tqdm
import plotly.express as px

from sentence_transformers import SentenceTransformer
from datasets import load_metric
from torch import nn

### Obtaining human judgements from the WMT data

In [ ]:
df_trans_thres = []
dfdev = pd.read_csv(f"{source_lang}_en_data/dev.{source_lang}en.df.short.tsv",sep='\t')
dftest = pd.read_csv(f"{source_lang}_en_data/test20.{source_lang}en.df.short.tsv", sep='\t')
frames = [dfdev, dftest]
final = pd.concat(frames)
comb = final.reset_index(drop=True)
devtestComb = comb.drop(columns=['index', 'mean', 'z_scores', 'z_mean', 'model_scores', 'scores'])

In [ ]:
df_orig_trans_score = devtestComb
####### BELOW CODE CALCULATES THE AVERAGE SCORE FOR HUMAN DA AND SUBSTITUTES IT IN THE TRANS_SCORE COLUMN INSTEAD OF THE PREDICTED SCORE #########
l = comb['scores'].tolist()
newlist = []
for sublist in l:
    sublist = sublist.split(", ")
    sublist = [i.strip('[]') for i in sublist]
    sublist = [float(i) for i in sublist]
    newlist.append(sublist)

newlistnparray = np.array(newlist)
newlistmeans = np.mean(newlistnparray, axis=1)
newlistmeans = newlistmeans/100
newlistmeans = newlistmeans.tolist()
df_orig_trans_score['trans_score_hum'] = newlistmeans
####### ABOVE CODE CALCULATES THE AVERAGE SCORE FOR HUMAN DA AND SUBSTITUTES IT IN THE TRANS_SCORE COLUMN INSTEAD OF THE PREDICTED SCORE #########

def trans_threshold(df, th=0.0):
  trans_score_np = np.array(list(df['trans_score_hum'])) ## CHANGE HERE BASED ON WHICH COLUMN SHOULD THE THRESHOLDING BE APPLIED
  trans_score_idx_ = np.argwhere(trans_score_np >= th)
  trans_score_idx = trans_score_idx_.reshape(trans_score_idx_.shape[0])

  return df.iloc[trans_score_idx]

trans_np = np.array(list(df_orig_trans_score['trans_score_hum']))

### Unsupervised method definition for obtaining scores for all sentences initially.

In [ ]:
df_trans_thres = trans_threshold(df_orig_trans_score, 0.0) 
df_trans_thres['ref'] = df_trans_thres['translation']
df_trans_thres['trans_score'] = df_trans_thres['trans_score_hum']
df_trans_thres = df_trans_thres.drop(columns=['trans_score_hum'])

In [ ]:
def combine_metrics(*args, **kwargs):
    assert len(args) == len(kwargs["corr"])
    assert len(args[0]) == len(args[1])
    
    output = []
    
    for i in range(len(args[0])):
        value = 0
        for sign, metric in zip(kwargs["corr"],args):
            if sign > 0:
                value += np.exp(metric[i])
            else:
                value += np.exp(1-metric[i])
        output.append(value)
    return output

bert_score_metric = load_metric('bertscore', keep_in_memory=False, cache_dir=sys.path[0])
bert_score_model = 'xlm-roberta-base'
cos_sim_model = SentenceTransformer('xlm-r-bert-base-nli-stsb-mean-tokens')

### Applying threshold of 0.7 now

In [ ]:
df_trans_thres = trans_threshold(df_orig_trans_score, 0.7) #Selection of threshold 0.7
df_trans_thres['ref'] = df_trans_thres['translation']
df_trans_thres['trans_score'] = df_trans_thres['trans_score_hum']
df_trans_thres = df_trans_thres.drop(columns=['trans_score_hum'])

In [ ]:
def predict_scores_orig(df, nTrials=1):
    ## Need a temporary scoring function here. 
    scores = np.zeros((1, df.shape[0]))
  
    for i in tqdm(range(nTrials)):
        embed_sent1 = cos_sim_model.encode(df['translation'].values.tolist(), convert_to_tensor=True)
        embed_sent2 = cos_sim_model.encode(df['original'].values.tolist(), convert_to_tensor=True)
        cos_sim = nn.CosineSimilarity(dim=1)(embed_sent1,embed_sent2)
        cos_sim = (cos_sim -torch.min(cos_sim))/ (torch.max(cos_sim)-torch.min(cos_sim))
        cos_scores = cos_sim.cpu().numpy()

        bert_score_metric.add_batch(predictions=df['translation'].values.tolist(), references=df['original'].values.tolist())
        score = bert_score_metric.compute(model_type=bert_score_model)
        norm_score = (score["f1"] - np.min(score["f1"])) / (np.max(score["f1"]) - np.min(score["f1"]))
        compound_metric = combine_metrics(cos_scores, norm_score, corr=[1, 1])
        df_norm = pd.DataFrame()
        df_norm['scores'] = compound_metric
        df_norm['scores'] = (df_norm['scores'] - df_norm['scores'].min()) / (df_norm['scores'].max() - df_norm['scores'].min())
        predictions = df_norm['scores'].values.tolist()
    
        scores[i] = predictions

    return scores

In [ ]:
scores_orig=[]
scores_orig = predict_scores_orig(df_trans_thres)
df_trans_thres.drop(columns='trans_score')
df_trans_thres['trans_score'] = scores_orig[0]

### Vocbulary generation from the preprocessed data

In [ ]:
class Vocabulary:
    def __init__(self, name):
        PAD_token = 0   # Used for padding short sentences
        SOS_token = 1   # Start-of-sentence token
        EOS_token = 2   # End-of-sentence token
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3
        self.num_sentences = 0
        self.longest_sentence = 0

    def add_word(self, word):
        if word not in self.word2index:
            # First entry of word into vocabulary
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            # Word exists; increase word count
            self.word2count[word] += 1
            
    def add_sentence(self, sentence):
        sentence_len = 0
        for word in sentence.split(' '):
            sentence_len += 1
            self.add_word(word)
        if sentence_len > self.longest_sentence:
            # This is the longest sentence
            self.longest_sentence = sentence_len
        # Count the number of sentences
        self.num_sentences += 1

    def to_word(self, index):
        return self.index2word[index]

    def to_index(self, word):
        return self.word2index[word]

In [ ]:
voc = Vocabulary('corpusVocab')
for sentence in devtestComb['translation']:
    voc.add_sentence(sentence.translate(str.maketrans('', '', string.punctuation))) ## removing puntuations

# vocab with stopwords 
vocab = []
for word in range(voc.num_words):
    vocab.append(voc.to_word(word))
# vocab without stopwords 
vocab_wo_stopw = []
for word in vocab:
    if(word.lower() not in STOP_WORDS):
        vocab_wo_stopw.append(word)

len(vocab), len(vocab_wo_stopw)

## Perturbations and Scoring

In [ ]:
def generate_map1_sents(df, nTrials):
    ## returns the list of perturbed translation based on the removal of negation markers - "not, no, and n't"
    print("Generating sentences for MAP1")
    new_sents_lol = [] 
    map1_orig_sents_lol = []
    for i in range(nTrials):
        new_sents  = []
        map1_orig_sents = []
        for original, sentence in zip(df['original'], df['translation']):
            tempSentVoc = sentence.split()

            sentVoc = []
            for token in tempSentVoc:
                token = token.replace("n't", "", 1)
                sentVoc.append(token)
            
            sentVoc = [token for token in sentVoc if (token.lower() != "not" and token.lower() != "no")]
            
            
            if(sentence != ' '.join(sentVoc)):
                tempL=[]
                tempL.append(original)
                tempL.append(' '.join(sentVoc))
                new_sents.append(tempL)
                tempL=[]
                tempL.append(original)
                tempL.append(sentence)
                map1_orig_sents.append(tempL)
        new_sents_lol.append(new_sents)
        map1_orig_sents_lol.append(map1_orig_sents)

    return new_sents_lol, map1_orig_sents_lol

In [ ]:
def generate_map2_sents(df, nTrials):
    ## returns the list of perturbed translations based on removal of random content words
    print("Generating sentences for MAP2")
    puncts = list(set(string.punctuation))
    new_sents_lol = [] 
    vocab_ = vocab_wo_stopw
    for i in range(nTrials):
        new_sents  = []
        for sentence in df['translation']:
            sentVoc = [str(i) for i in tokenizer(sentence)]
            sentVoc_wo_punct = []

            for v in sentVoc:
                if(v not in puncts):
                    sentVoc_wo_punct.append(v)

            sentVoc = sentVoc_wo_punct
            
            sentVoc_wo_stop_word = []
            for v in sentVoc:
                if(v.lower() not in STOP_WORDS):
                    sentVoc_wo_stop_word.append(v)
            
            sentVoc = sentVoc_wo_stop_word
       
            new_word = "" 
            old_word = random.choice(sentVoc)
          
            flag = 0
            while(flag == 0):
                if(new_word not in sentVoc):
                    new_sents.append(sentence.replace(old_word, new_word, 1))
                    flag = 1
                else:
                    new_word = random.choice(vocab_)
    
        new_sents_lol.append(new_sents)
    return new_sents_lol

In [ ]:
def generate_map3_sents(df, nTrials):
    ## returns the list of perturbed translations based on duplication of a random content word
    print("Generating sentences for MAP3")
    puncts = list(set(string.punctuation))
    new_sents_lol = []
    vocab_ = vocab_wo_stopw
    for i in range(nTrials):
        new_sents  = []
        for sentence in df['translation']:
            sentVoc = [str(i) for i in tokenizer(sentence)]
            sentVoc_wo_punct = []

            for v in sentVoc:
                if(v not in puncts):
                    sentVoc_wo_punct.append(v)
            sentVoc = sentVoc_wo_punct

            sentVoc_wo_stop_word = []
            for v in sentVoc:
                if(v.lower() not in STOP_WORDS):
                    sentVoc_wo_stop_word.append(v)
            sentVoc = sentVoc_wo_stop_word
        
            old_word = random.choice(sentVoc)
            new_word = old_word + " " + old_word 
          
            flag = 0
            while (flag == 0):
                if(new_word not in sentVoc):
                    new_sents.append(sentence.replace(old_word, new_word, 1))
                    flag = 1
                else:
                    new_word = random.choice(vocab_)
    
        new_sents_lol.append(new_sents)
    return new_sents_lol

In [ ]:
def generate_map4_sents(df, nTrials):
    ## returns the list of perturbed translations based on insertion of random content word
    print("Generating sentences for MAP4")
    puncts = list(set(string.punctuation))
    new_sents_lol = []
    vocab_ = vocab_wo_stopw
    for i in range(nTrials):
        new_sents  = []
        for sentence in df['translation']:
            sentVoc = [str(i) for i in tokenizer(sentence)]
            sentVoc_wo_punct = []

            for v in sentVoc:
                if(v not in puncts):
                    sentVoc_wo_punct.append(v)
            sentVoc = sentVoc_wo_punct

            sentVoc_wo_stop_word = []
            for v in sentVoc:
                if(v.lower() not in STOP_WORDS):
                    sentVoc_wo_stop_word.append(v)
            sentVoc = sentVoc_wo_stop_word
        
            new_word = random.choice(vocab_)
            old_word = random.choice(sentVoc)
            new_word = old_word + " " + new_word 
          
            flag = 0
            while (flag == 0):
                if(new_word not in sentVoc):
                    new_sents.append(sentence.replace(old_word, new_word, 1))
                    flag = 1
                else:
                    new_word = random.choice(vocab_)
    
        new_sents_lol.append(new_sents)
    return new_sents_lol

In [ ]:
def generate_map5_sents(df, nTrials):
    ## returns the list of perturbed translations based on random replacements of words
    print("Generating sentences for MAP5")
    puncts = list(set(string.punctuation))
    new_sents_lol = []
    vocab_ = vocab
    for i in range(nTrials):
        new_sents  = []
        for sentence in df['translation']:
            sentVoc = [str(i) for i in tokenizer(sentence)]
            sentVoc_wo_punct = []

            for v in sentVoc:
                if(v not in puncts):
                    sentVoc_wo_punct.append(v)
            sentVoc = sentVoc_wo_punct

            sentVoc_wo_stop_word = []
            for v in sentVoc:
                if(v.lower() not in STOP_WORDS):
                    sentVoc_wo_stop_word.append(v)
            sentVoc = sentVoc_wo_stop_word
            
            new_word = random.choice(vocab_)
            old_word = random.choice(sentVoc)
          
            flag = 0
            while (flag == 0):
                if(new_word not in sentVoc):
                    new_sents.append(sentence.replace(old_word, new_word, 1))
                    flag = 1
                else:
                    new_word = random.choice(vocab_)
    
        new_sents_lol.append(new_sents)
    return new_sents_lol

In [ ]:
## For NLP Augmentation Library ##
# import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
# import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from nlpaug.util import Action
## For NLP Augmentation Library ##

In [ ]:
## For NLP Augmentation Library ##

## This takes time ##

aug = naw.ContextualWordEmbsAug( model_path='bert-base-uncased', action="substitute" )

## For NLP Augmentation Library ##

In [ ]:
## This function uses the NLP Augmentation Library##

def generate_map6_sents(df, nTrials):
    ## returns the list of perturbed translations based on synonymy replacement using BERT, considered an MAP as BERT-based replacements
    ## have been shown to replace important content words, which do change the meaning.
    print("Generating sentences for MAP6 - This takes some time for all the 20 trials.")
    new_sents_lol = []
    map6_orig_sents_lol = []
    for i in range(nTrials):
        new_sents  = []
        map6_orig_sents = []

        for original, sentence in zip(df['original'], df['translation']):    
            origSent = sentence.split()
            augmented_text = aug.augment(sentence)

            if((' '.join(origSent)) != augmented_text):
                tempL=[]
                tempL.append(original)
                tempL.append(augmented_text)
                new_sents.append(tempL)
                tempL=[]
                tempL.append(original)
                tempL.append(' '.join(origSent))
                map6_orig_sents.append(tempL)
         
        new_sents_lol.append(new_sents)
        map6_orig_sents_lol.append(map6_orig_sents)
    
    return new_sents_lol, map6_orig_sents_lol

In [ ]:
augAntonym = naw.AntonymAug()

def generate_map7_sents(df, nTrials, stopwords=True):
    ## returns the list of perturbed translations where words are replace by their Antonyms
    print("Generating sentences for MAP7")
    new_sents_lol = [] 
    map7_orig_sents_lol = []
    for i in range(nTrials):
        new_sents  = []
        map7_orig_sents = []

        for original, sentence in zip(df['original'], df['translation']):     
            origSent = sentence.split()
            augmented_text = augAntonym.augment(sentence)
            
            if((' '.join(origSent)) != augmented_text):  
                tempL=[]
                tempL.append(original)
                tempL.append(augmented_text)
                new_sents.append(tempL)
                tempL=[]
                tempL.append(original)
                tempL.append(' '.join(origSent))
                map7_orig_sents.append(tempL)
        new_sents_lol.append(new_sents)
        map7_orig_sents_lol.append(map7_orig_sents)

    return new_sents_lol, map7_orig_sents_lol

In [ ]:
def generate_map8_sents(df, nTrials):
    # returns a list of perturbed translations where it copyies source sentence instead of target
    print("Generating sentences for MAP8")
    new_sents_lol = [] 
    for i in range(nTrials):
        new_sents  = []
        for sentence in df['original']:      
            new_sents.append(sentence)
        new_sents_lol.append(new_sents)

    return new_sents_lol

### Generating sentences for MAPs

In [ ]:
new_sents_lol_map1, map1_orig_sents = generate_map1_sents(df_trans_thres, 20) ## Perturbations for removing negation markers
new_sents_lol_map2  = generate_map2_sents(df_trans_thres, 20) ## Perturbations for removing random content words
new_sents_lol_map3  = generate_map3_sents(df_trans_thres, 20) ## Perturbations for duplicating random content words
new_sents_lol_map4  = generate_map4_sents(df_trans_thres, 20) ## Perturbations for inserting random content words
new_sents_lol_map5  = generate_map5_sents(df_trans_thres, 20) ## Perturbations for replacing random content words

## For MAP6, each trial takes ~1 minute over 16 core machine. This library uses CPU cores, and you are passing 20 trials, so this takes time.

new_sents_lol_map6, map6_orig_sents = generate_map6_sents(df_trans_thres, 20) ## Perturbations for replacing target by source

## For MAP6, each trial takes ~1 minute over 16 core machine. This library uses CPU cores, and you are passing 20 trials, so this takes time.

new_sents_lol_map7, map7_orig_sents = generate_map7_sents(df_trans_thres, 20) ## Perturbationeachs for replacing words by their Antonyms
new_sents_lol_map8  = generate_map8_sents(df_trans_thres, 20) ## Perturbations for replacing translation with source sentence

In [ ]:
def generate_mpp1_sents(df, nTrials):
    ##returns the list of perturbed translations based removal of punctuations
    print("Generating sentences for MPP1")
    exclude = set(string.punctuation)
    new_sents_lol = [] 
    mpp1_orig_sents_lol = []
    
    for i in range(nTrials):
        new_sents  = []
        mpp1_orig_sents = []

        for original, sentence in zip(df['original'], df['translation']):
            newSentence = ''.join(ch for ch in sentence if ch not in exclude)
            sentVoc = newSentence.split()
            origSent = sentence.split()
      
            if((' '.join(origSent)) != (' '.join(sentVoc))):
                tempL=[]
                tempL.append(original)
                tempL.append(' '.join(sentVoc))
                new_sents.append(tempL)
                tempL=[]
                tempL.append(original)
                tempL.append(sentence)
                mpp1_orig_sents.append(tempL)
        
        new_sents_lol.append(new_sents)
        mpp1_orig_sents_lol.append(mpp1_orig_sents)
    
    return new_sents_lol, mpp1_orig_sents_lol

In [ ]:
def generate_mpp2_sents(df, nTrials):
    ## returns the list of perturbed translations based on replacement of punctuations
    print("Generating sentences for MPP2")
    puncts = list(set(string.punctuation))
    new_sents_lol = [] 
    mpp2_orig_sents_lol = []

    for i in range(nTrials):
        new_sents  = []
        mpp2_orig_sents = []

        for original, sentence in zip(df['original'], df['translation']):
            for i in range(len(sentence)):
                if(sentence[i] in puncts): #Found punctuation, replacing here.
                    new_punct = random.choice(puncts)
                    if(new_punct!=sentence[i]):
                        newSentence = sentence.replace(sentence[i], new_punct)
            sentVoc = newSentence.split()
            origSent = sentence.split()
      
            if((' '.join(origSent)) != (' '.join(sentVoc))):
                tempL=[]
                tempL.append(original)
                tempL.append(' '.join(sentVoc))
                new_sents.append(tempL)
                tempL=[]
                tempL.append(original)
                tempL.append(sentence)
                mpp2_orig_sents.append(tempL)
                
        new_sents_lol.append(new_sents)
        mpp2_orig_sents_lol.append(mpp2_orig_sents)
    
    return new_sents_lol, mpp2_orig_sents_lol

In [ ]:
## Required for MPP3/4 ##
nlp = spacy.load('en_core_web_sm')
## Required for MPP3/4 ##

In [ ]:
def generate_mpp3_sents(df, nTrials):
    ## returns the list of perturbed translations based on removal of determiners
    print("Generating sentences for MPP3")
    puncts = list(set(string.punctuation))
    new_sents_lol = []
    mpp3_orig_sents_lol = []
    
    for i in range(nTrials):
        new_sents  = []
        mpp3_orig_sents = []

        for original, sentence in zip(df['original'], df['translation']):
            doc = nlp(sentence)
            sentVoc = [token.text for token in doc if token.pos_ != "DET"]
            origSent = sentence.split()
            indices = []
            for i in range(len(sentVoc)):
                if(sentVoc[i] in puncts):
                    sentVoc[i-1] = sentVoc[i-1]+sentVoc[i]
                    indices.append(i)
            iter=0
            for idx in indices:
                sentVoc.pop(idx-iter)
                iter+=1
            if((' '.join(origSent)) != (' '.join(sentVoc))):
                tempL=[]
                tempL.append(original)
                tempL.append(' '.join(sentVoc))
                new_sents.append(tempL)
                tempL=[]
                tempL.append(original)
                tempL.append(sentence)
                mpp3_orig_sents.append(tempL)
        new_sents_lol.append(new_sents)
        mpp3_orig_sents_lol.append(mpp3_orig_sents)
    
    return new_sents_lol, mpp3_orig_sents_lol

In [ ]:
def generate_mpp4_sents(df, nTrials):
    ## returns the list of perturbed translations based on replacement of determiners from a list
    print("Generating sentences for MPP4")
    detList =  ["the", "a", "an", "this", "that", "these", "those", "my", "your", "his", "her", "its", "our", "their", "some", "any", "enough", "all", "each", "every", "other", "another", "such"]
    new_sents_lol = [] 
    mpp4_orig_sents_lol = []

    for i in range(nTrials):
        new_sents  = []
        mpp4_orig_sents = []

        for original, sentence in zip(df['original'], df['translation']):
            doc = nlp(sentence)
            sentVoc = []
            origSent = sentence.split()
            for i in range(len(doc)):
                if(doc[i].pos_ == "DET"):
                    sentVoc.append(random.choice(detList))
                else:
                    sentVoc.append(str(doc[i]))

            if((' '.join(origSent)) != (' '.join(sentVoc))):  
                tempL=[]
                tempL.append(original)
                tempL.append(' '.join(sentVoc))
                new_sents.append(tempL)
                tempL=[]
                tempL.append(original)
                tempL.append(sentence)
                mpp4_orig_sents.append(tempL)

        new_sents_lol.append(new_sents)
        mpp4_orig_sents_lol.append(mpp4_orig_sents)
    
    return new_sents_lol, mpp4_orig_sents_lol

In [ ]:
def generate_mpp5_sents(df, nTrials):
    ## returns the list of perturbed translations based on uppercasing words
    print("Generating sentences for MPP5")
    new_sents_lol = []

    for i in range(nTrials):
        new_sents  = []
        for sentence in df['translation']:
            sentVoc = sentence.split()
            for i in range(len(sentVoc)):
                wordswithUC = []
                wordswithUC.append(str(sentVoc[i]))
                wordswithUC.append(str(sentVoc[i].upper()))
                sentVoc[i] = random.choice(wordswithUC)
          
            new_sents.append(' '.join(sentVoc))
        new_sents_lol.append(new_sents)

    return new_sents_lol

In [ ]:
def generate_mpp6_sents(df, nTrials):
    ## returns the list of perturbed translations based on lowercasing words
    print("Generating sentences for MPP6")
    new_sents_lol = []

    for i in range(nTrials):
        new_sents  = []
        for sentence in df['translation']:
            sentVoc = sentence.split()
            for i in range(len(sentVoc)):
                wordswithUC = []
                wordswithUC.append(str(sentVoc[i]))
                wordswithUC.append(str(sentVoc[i].lower()))
                sentVoc[i] = random.choice(wordswithUC)
          
            new_sents.append(' '.join(sentVoc))
        new_sents_lol.append(new_sents)

    return new_sents_lol

### Generating sentences for MPPs

In [ ]:
new_sents_lol_mpp1, mpp1_orig_sents = generate_mpp1_sents(df_trans_thres, 20) ## Perturbations for removing punctuations
new_sents_lol_mpp2, mpp2_orig_sents = generate_mpp2_sents(df_trans_thres, 20) ## Perturbations for replacing punctuations
new_sents_lol_mpp3, mpp3_orig_sents = generate_mpp3_sents(df_trans_thres, 20) ## Perturbations for removing determiners
new_sents_lol_mpp4, mpp4_orig_sents = generate_mpp4_sents(df_trans_thres, 20) ## Perturbations for replacing determiners
new_sents_lol_mpp5 = generate_mpp5_sents(df_trans_thres, 20) ## Perturbations for random uppercasing
new_sents_lol_mpp6 = generate_mpp6_sents(df_trans_thres, 20) ## Perturbations for random lowercasing

In [ ]:
def predict_scores_1(df, new_sents, nTrials):
  scores = np.zeros((nTrials + 1 , df.shape[0]))
  scores[0] = df['trans_score'].to_numpy() # storing prediction for original translations at index 0
  
  for i in tqdm(range(nTrials)):
    embed_sent1 = cos_sim_model.encode(new_sents[i], convert_to_tensor=True)
    embed_sent2 = cos_sim_model.encode(df['original'].values.tolist(), convert_to_tensor=True)
    cos_sim = nn.CosineSimilarity(dim=1)(embed_sent1,embed_sent2)
    cos_sim = (cos_sim -torch.min(cos_sim))/ (torch.max(cos_sim)-torch.min(cos_sim))
    cos_scores = cos_sim.cpu().numpy()

    bert_score_metric.add_batch(predictions=new_sents[i], references=df['original'].values.tolist())
    score = bert_score_metric.compute(model_type=bert_score_model)
    norm_score = (score["f1"] - np.min(score["f1"])) / (np.max(score["f1"]) - np.min(score["f1"]))

    compound_metric = combine_metrics(cos_scores, norm_score, corr=[1, 1])
    df_norm = pd.DataFrame()
    df_norm['scores'] = compound_metric
    df_norm['scores'] = (df_norm['scores'] - df_norm['scores'].min()) / (df_norm['scores'].max() - df_norm['scores'].min())
    predictions = df_norm['scores'].values.tolist()
    
    scores[1+i] = predictions

  return scores

In [ ]:
def predict_scores_2(df, orig_list_lol, new_sents_lol, nTrials):
    x = len(orig_list_lol)
    y = max(map(len, orig_list_lol))
    scores_orig = np.zeros([x,y])
    sources=[]
    trans=[]
    for i in range(len(orig_list_lol)):
        temp_sources=[]
        temp_trans=[]
        for j in range(len(orig_list_lol[i])):
            temp_sources.append(orig_list_lol[i][j][0])
            temp_trans.append(orig_list_lol[i][j][1])
        sources.append(temp_sources)
        trans.append(temp_trans)
    for i in range(len(sources)):
        sent_score=[]
        for j in range(len(sources[i])):
            counter = 0
            for dftrans, score in zip(df['original'], df['trans_score']):
                if(str(sources[i][j]) == str(dftrans)):
                    sent_score.append(score)
                    counter = 1
                    break;
            if(counter == 0):
                print("NF: " + trans[i][j])
        scores_orig[i,:len(sent_score)] = sent_score
    print("Done with scores for original Sentence pairs - Trial "+str(i+1))
    x = len(new_sents_lol)
    y = max(map(len, new_sents_lol))
    scores_pert = np.zeros([x,y])
    sources=[]
    trans=[]
    for i in range(len(new_sents_lol)):
        temp_sources=[]
        temp_trans=[]
        for j in range(len(new_sents_lol[i])):
            temp_sources.append(new_sents_lol[i][j][0])
            temp_trans.append(new_sents_lol[i][j][1])
        sources.append(temp_sources)
        trans.append(temp_trans)
    for j in range(len(new_sents_lol)):
        embed_sent1 = cos_sim_model.encode(trans[i], convert_to_tensor=True)
        embed_sent2 = cos_sim_model.encode(sources[i], convert_to_tensor=True)
        cos_sim = nn.CosineSimilarity(dim=1)(embed_sent1,embed_sent2)
        cos_sim = (cos_sim -torch.min(cos_sim))/ (torch.max(cos_sim)-torch.min(cos_sim))
        cos_scores = cos_sim.cpu().numpy()
  
        bert_score_metric.add_batch(predictions=trans[i], references=sources[i])
        score = bert_score_metric.compute(model_type=bert_score_model)
        norm_score = (score["f1"] - np.min(score["f1"])) / (np.max(score["f1"]) - np.min(score["f1"]))
        p1predictions = combine_metrics(cos_scores, norm_score, corr=[1, 1])
        df_norm = pd.DataFrame()
        df_norm['scores'] = p1predictions
        df_norm['scores'] = (df_norm['scores'] - df_norm['scores'].min()) / (df_norm['scores'].max() - df_norm['scores'].min())
        p2predictions = df_norm['scores'].values.tolist()
        scores_pert[j,:len(p2predictions)] = p2predictions
        if(j==19):
            print("Done with scores for perturbed sentence pairs - Trial "+str(j+1))
        
    return scores_orig, scores_pert

### Generating scores and dumping them in files

In [ ]:
## This will take some time, score computation uses the models to get scores for each sentence pair in each trial  ##

scores_map1_orig, scores_map1_perts = predict_scores_2(df_trans_thres, map1_orig_sents, new_sents_lol_map1, 20)
scores_map2 = predict_scores_1(df_trans_thres, new_sents_lol_map2, 20)
scores_map3 = predict_scores_1(df_trans_thres, new_sents_lol_map3, 20)
scores_map4 = predict_scores_1(df_trans_thres, new_sents_lol_map4, 20)
scores_map5 = predict_scores_1(df_trans_thres, new_sents_lol_map5, 20)
scores_map6_orig, scores_map6_perts = predict_scores_2(df_trans_thres, map6_orig_sents, new_sents_lol_map6, 20)
scores_map7_orig, scores_map7_perts = predict_scores_2(df_trans_thres, map7_orig_sents, new_sents_lol_map7, 20)
scores_map8 = predict_scores_1(df_trans_thres, new_sents_lol_map8, 20)

## This will take some time, score computation uses the models to get scores for each sentence pair in each trial  ##

In [ ]:
!mkdir -p generated_data_{source_lang}_en

In [ ]:
################## MAP1 - Dumping the score into a dataframe for computation and in a CSV file for retrieving later ##################
columnsList = []
for i in range(len(scores_map1_perts)):
    columnsList.append('pert'+str(i+1))

scores_map1_perts_df = pd.DataFrame(list(map(list, zip(*scores_map1_perts))), columns = columnsList)
scores_map1_perts_df.to_csv('generated_data_'+source_lang+'_en/scores_map1_perts_df_'+tqmodel+'.csv', index = False, sep=';')

columnsList = []
for i in range(len(scores_map1_orig)):
    columnsList.append('orig'+str(i+1))

scores_map1_orig_df = pd.DataFrame(list(map(list, zip(*scores_map1_orig))), columns = columnsList)
scores_map1_orig_df.to_csv('generated_data_'+source_lang+'_en/scores_map1_orig_df_'+tqmodel+'.csv', index = False, sep=';')
######################################################################################################################################

################## MAP2 - Dumping the score into a dataframe for computation and in a CSV file for retrieving later ##################
columnsList = []
for i in range(len(scores_map2)):
    columnsList.append('pert'+str(i+1))

scores_map2_df = pd.DataFrame(list(map(list, zip(*scores_map2))), columns = columnsList)
scores_map2_df.to_csv('generated_data_'+source_lang+'_en/scores_map2_df_'+tqmodel+'.csv', index = False, sep=';')
######################################################################################################################################

################## MAP3 - Dumping the score into a dataframe for computation and in a CSV file for retrieving later ##################
columnsList = []
for i in range(len(scores_map3)):
    columnsList.append('pert'+str(i+1))

scores_map3_df = pd.DataFrame(list(map(list, zip(*scores_map3))), columns = columnsList)
scores_map3_df.to_csv('generated_data_'+source_lang+'_en/scores_map3_df_'+tqmodel+'.csv', index = False, sep=';')

######################################################################################################################################

################## MAP4 - Dumping the score into a dataframe for computation and in a CSV file for retrieving later ##################
columnsList = []
for i in range(len(scores_map4)):
    columnsList.append('pert'+str(i+1))

scores_map4_df = pd.DataFrame(list(map(list, zip(*scores_map4))), columns = columnsList)
scores_map4_df.to_csv('generated_data_'+source_lang+'_en/scores_map4_df_'+tqmodel+'.csv', index = False, sep=';')
######################################################################################################################################

################## MAP5 - Dumping the score into a dataframe for computation and in a CSV file for retrieving later ##################
columnsList = []
for i in range(len(scores_map5)):
    columnsList.append('pert'+str(i+1))

scores_map5_df = pd.DataFrame(list(map(list, zip(*scores_map5))), columns = columnsList)
scores_map5_df.to_csv('generated_data_'+source_lang+'_en/scores_map5_df_'+tqmodel+'.csv', index = False, sep=';')
######################################################################################################################################

################## MAP6 - Dumping the score into a dataframe for computation and in a CSV file for retrieving later ##################
columnsList = []
for i in range(len(scores_map6_perts)):
    columnsList.append('pert'+str(i+1))

scores_map6_perts_df = pd.DataFrame(list(map(list, zip(*scores_map6_perts))), columns = columnsList)
scores_map6_perts_df.to_csv('generated_data_'+source_lang+'_en/scores_map6_perts_df_'+tqmodel+'.csv', index = False, sep=';')

columnsList = []
for i in range(len(scores_map6_orig)):
    columnsList.append('orig'+str(i+1))

scores_map6_orig_df = pd.DataFrame(list(map(list, zip(*scores_map6_orig))), columns = columnsList)
scores_map6_orig_df.to_csv('generated_data_'+source_lang+'_en/scores_map6_orig_df_'+tqmodel+'.csv', index = False, sep=';')
######################################################################################################################################

################## MAP7 - Dumping the score into a dataframe for computation and in a CSV file for retrieving later ##################
columnsList = []
for i in range(len(scores_map7_perts)):
    columnsList.append('pert'+str(i+1))

scores_map7_perts_df = pd.DataFrame(list(map(list, zip(*scores_map7_perts))), columns = columnsList)
scores_map7_perts_df.to_csv('generated_data_'+source_lang+'_en/scores_map7_perts_df_'+tqmodel+'.csv', index = False, sep=';')

columnsList = []
for i in range(len(scores_map7_orig)):
    columnsList.append('orig'+str(i+1))

scores_map7_orig_df = pd.DataFrame(list(map(list, zip(*scores_map7_orig))), columns = columnsList)
scores_map7_orig_df.to_csv('generated_data_'+source_lang+'_en/scores_map7_orig_df_'+tqmodel+'.csv', index = False, sep=';')
######################################################################################################################################

################## MAP8 - Dumping the score into a dataframe for computation and in a CSV file for retrieving later ##################
columnsList = []
for i in range(len(scores_map8)):
    columnsList.append('pert'+str(i+1))

scores_map8_df = pd.DataFrame(list(map(list, zip(*scores_map8))), columns = columnsList)
scores_map8_df.to_csv('generated_data_'+source_lang+'_en/scores_map8_df_'+tqmodel+'.csv', index = False, sep=';')
######################################################################################################################################

In [ ]:
## This will take some time, score computation uses the models to get scores for each sentence pair in each trial  ##

scores_mpp1_orig, scores_mpp1_perts = predict_scores_2(df_trans_thres, mpp1_orig_sents, new_sents_lol_mpp1, 20)
scores_mpp2_orig, scores_mpp2_perts = predict_scores_2(df_trans_thres, mpp2_orig_sents, new_sents_lol_mpp2, 20)
scores_mpp3_orig, scores_mpp3_perts = predict_scores_2(df_trans_thres, mpp3_orig_sents, new_sents_lol_mpp3, 20)
scores_mpp4_orig, scores_mpp4_perts = predict_scores_2(df_trans_thres, mpp4_orig_sents, new_sents_lol_mpp4, 20)

scores_mpp5 = predict_scores_1(df_trans_thres, new_sents_lol_mpp5, 20)
scores_mpp6 = predict_scores_1(df_trans_thres, new_sents_lol_mpp6, 20)

## This will take some time, score computation uses the models to get scores for each sentence pair in each trial  ##

In [ ]:
################## MPP1 - Dumping the score into a dataframe for computation and in a CSV file for retrieving later ##################
columnsList = []
for i in range(len(scores_mpp1_perts)):
    columnsList.append('pert'+str(i+1))

scores_mpp1_perts_df = pd.DataFrame(list(map(list, zip(*scores_mpp1_perts))), columns = columnsList)
scores_mpp1_perts_df.to_csv('generated_data_'+source_lang+'_en/scores_mpp1_perts_df_'+tqmodel+'.csv', index = False, sep=';')

columnsList = []
for i in range(len(scores_mpp1_orig)):
    columnsList.append('orig'+str(i+1))

scores_mpp1_orig_df = pd.DataFrame(list(map(list, zip(*scores_mpp1_orig))), columns = columnsList)
scores_mpp1_orig_df.to_csv('generated_data_'+source_lang+'_en/scores_mpp1_orig_df_'+tqmodel+'.csv', index = False, sep=';')
######################################################################################################################################

################## MPP2 - Dumping the score into a dataframe for computation and in a CSV file for retrieving later ##################
columnsList = []
for i in range(len(scores_mpp2_perts)):
    columnsList.append('pert'+str(i+1))

scores_mpp2_perts_df = pd.DataFrame(list(map(list, zip(*scores_mpp2_perts))), columns = columnsList)
scores_mpp2_perts_df.to_csv('generated_data_'+source_lang+'_en/scores_mpp2_perts_df_'+tqmodel+'.csv', index = False, sep=';')

columnsList = []
for i in range(len(scores_mpp2_orig)):
    columnsList.append('orig'+str(i+1))

scores_mpp2_orig_df = pd.DataFrame(list(map(list, zip(*scores_mpp2_orig))), columns = columnsList)
scores_mpp2_orig_df.to_csv('generated_data_'+source_lang+'_en/scores_mpp2_orig_df_'+tqmodel+'.csv', index = False, sep=';')
######################################################################################################################################

################## MPP3 - Dumping the score into a dataframe for computation and in a CSV file for retrieving later ##################
columnsList = []
for i in range(len(scores_mpp3_perts)):
    columnsList.append('pert'+str(i+1))

scores_mpp3_perts_df = pd.DataFrame(list(map(list, zip(*scores_mpp3_perts))), columns = columnsList)
scores_mpp3_perts_df.to_csv('generated_data_'+source_lang+'_en/scores_mpp3_perts_df_'+tqmodel+'.csv', index = False, sep=';')

columnsList = []
for i in range(len(scores_mpp3_orig)):
    columnsList.append('orig'+str(i+1))

scores_mpp3_orig_df = pd.DataFrame(list(map(list, zip(*scores_mpp3_orig))), columns = columnsList)
scores_mpp3_orig_df.to_csv('generated_data_'+source_lang+'_en/scores_mpp3_orig_df_'+tqmodel+'.csv', index = False, sep=';')
######################################################################################################################################

################## MPP4 - Dumping the score into a dataframe for computation and in a CSV file for retrieving later ##################
columnsList = []
for i in range(len(scores_mpp4_perts)):
    columnsList.append('pert'+str(i+1))

scores_mpp4_perts_df = pd.DataFrame(list(map(list, zip(*scores_mpp4_perts))), columns = columnsList)
scores_mpp4_perts_df.to_csv('generated_data_'+source_lang+'_en/scores_mpp4_perts_df_'+tqmodel+'.csv', index = False, sep=';')

columnsList = []
for i in range(len(scores_mpp4_orig)):
    columnsList.append('orig'+str(i+1))

scores_mpp4_orig_df = pd.DataFrame(list(map(list, zip(*scores_mpp4_orig))), columns = columnsList)
scores_mpp4_orig_df.to_csv('generated_data_'+source_lang+'_en/scores_mpp4_orig_df_'+tqmodel+'.csv', index = False, sep=';')
######################################################################################################################################

################## MPP5 - Dumping the score into a dataframe for computation and in a CSV file for retrieving later ##################
columnsList = []
for i in range(len(scores_mpp5)):
    columnsList.append('pert'+str(i+1))

scores_mpp5_df = pd.DataFrame(list(map(list, zip(*scores_mpp5))), columns = columnsList)
scores_mpp5_df.to_csv('generated_data_'+source_lang+'_en/scores_mpp5_df_'+tqmodel+'.csv', index = False, sep=';')
######################################################################################################################################

################## MPP6 - Dumping the score into a dataframe for computation and in a CSV file for retrieving later ##################
columnsList = []
for i in range(len(scores_mpp6)):
    columnsList.append('pert'+str(i+1))

scores_mpp6_df = pd.DataFrame(list(map(list, zip(*scores_mpp6))), columns = columnsList)
scores_mpp6_df.to_csv('generated_data_'+source_lang+'_en/scores_mpp6_df_'+tqmodel+'.csv', index = False, sep=';')
######################################################################################################################################

### Once the scores are generated for a particular language pair, use these two code blocks below to read the scores from files. 
### If redoing experiments, this can save the time required to regenerate the scores.

In [ ]:
scores_map1_perts_df = pd.read_csv('generated_data_'+source_lang+'_en/scores_map1_perts_df_'+tqmodel+'.csv', sep=';')
scores_map1_perts = scores_map1_perts_df.transpose().to_numpy()
scores_map1_orig_df = pd.read_csv('generated_data_'+source_lang+'_en/scores_map1_orig_df_'+tqmodel+'.csv', sep=';')
scores_map1_orig = scores_map1_orig_df.transpose().to_numpy()

scores_map2_df = pd.read_csv('generated_data_'+source_lang+'_en/scores_map2_df_'+tqmodel+'.csv', sep=';')
scores_map2 = scores_map2_df.transpose().to_numpy()

scores_map3_df = pd.read_csv('generated_data_'+source_lang+'_en/scores_map3_df_'+tqmodel+'.csv', sep=';')
scores_map3 = scores_map3_df.transpose().to_numpy()

scores_map3_df = pd.read_csv('generated_data_'+source_lang+'_en/scores_map3_df_'+tqmodel+'.csv', sep=';')
scores_map3 = scores_map3_df.transpose().to_numpy()

scores_map4_df = pd.read_csv('generated_data_'+source_lang+'_en/scores_map4_df_'+tqmodel+'.csv', sep=';')
scores_map4 = scores_map4_df.transpose().to_numpy()

scores_map5_df = pd.read_csv('generated_data_'+source_lang+'_en/scores_map5_df_'+tqmodel+'.csv', sep=';')
scores_map5 = scores_map5_df.transpose().to_numpy()

scores_map6_perts_df = pd.read_csv('generated_data_'+source_lang+'_en/scores_map6_perts_df_'+tqmodel+'.csv', sep=';')
scores_map6_perts = scores_map6_perts_df.transpose().to_numpy()
scores_map6_orig_df = pd.read_csv('generated_data_'+source_lang+'_en/scores_map6_orig_df_'+tqmodel+'.csv', sep=';')
scores_map6_orig = scores_map6_orig_df.transpose().to_numpy()

scores_map7_perts_df = pd.read_csv('generated_data_'+source_lang+'_en/scores_map7_perts_df_'+tqmodel+'.csv', sep=';')
scores_map7_perts = scores_map7_perts_df.transpose().to_numpy()
scores_map7_orig_df = pd.read_csv('generated_data_'+source_lang+'_en/scores_map7_orig_df_'+tqmodel+'.csv', sep=';')
scores_map7_orig = scores_map7_orig_df.transpose().to_numpy()

scores_map8_df = pd.read_csv('generated_data_'+source_lang+'_en/scores_map8_df_'+tqmodel+'.csv', sep=';')
scores_map8 = scores_map8_df.transpose().to_numpy()

In [ ]:
scores_mpp1_perts_df = pd.read_csv('generated_data_'+source_lang+'_en/scores_mpp1_perts_df_'+tqmodel+'.csv', sep=';')
scores_mpp1_perts = scores_mpp1_perts_df.transpose().to_numpy()
scores_mpp1_orig_df = pd.read_csv('generated_data_'+source_lang+'_en/scores_mpp1_orig_df_'+tqmodel+'.csv', sep=';')
scores_mpp1_orig = scores_mpp1_orig_df.transpose().to_numpy()

scores_mpp2_perts_df = pd.read_csv('generated_data_'+source_lang+'_en/scores_mpp2_perts_df_'+tqmodel+'.csv', sep=';')
scores_mpp2_perts = scores_mpp2_perts_df.transpose().to_numpy()
scores_mpp2_orig_df = pd.read_csv('generated_data_'+source_lang+'_en/scores_mpp2_orig_df_'+tqmodel+'.csv', sep=';')
scores_mpp2_orig = scores_mpp2_orig_df.transpose().to_numpy()

scores_mpp3_perts_df = pd.read_csv('generated_data_'+source_lang+'_en/scores_mpp3_perts_df_'+tqmodel+'.csv', sep=';')
scores_mpp3_perts = scores_mpp3_perts_df.transpose().to_numpy()
scores_mpp3_orig_df = pd.read_csv('generated_data_'+source_lang+'_en/scores_mpp3_orig_df_'+tqmodel+'.csv', sep=';')
scores_mpp3_orig = scores_mpp3_orig_df.transpose().to_numpy()

scores_mpp4_perts_df = pd.read_csv('generated_data_'+source_lang+'_en/scores_mpp4_perts_df_'+tqmodel+'.csv', sep=';')
scores_mpp4_perts = scores_mpp4_perts_df.transpose().to_numpy()
scores_mpp4_orig_df = pd.read_csv('generated_data_'+source_lang+'_en/scores_mpp4_orig_df_'+tqmodel+'.csv', sep=';')
scores_mpp4_orig = scores_mpp4_orig_df.transpose().to_numpy()

scores_mpp5_df = pd.read_csv('generated_data_'+source_lang+'_en/scores_mpp5_df_'+tqmodel+'.csv', sep=';')
scores_mpp5 = scores_mpp5_df.transpose().to_numpy()

scores_mpp6_df = pd.read_csv('generated_data_'+source_lang+'_en/scores_mpp6_df_'+tqmodel+'.csv', sep=';')
scores_mpp6 = scores_mpp6_df.transpose().to_numpy()

## Computing Relative Differences

In [ ]:
scores_mean_map1_perts = np.mean(scores_map1_perts[0:],axis=0) ## Mean of 20 trials of perturbed scores.
scores_mean_map1_orig = np.mean(scores_map1_orig[0:],axis=0) ## Mean of 20 trails of original translation scores
scores_relative_diff_map1 = scores_mean_map1_orig - scores_mean_map1_perts ## Computing relative difference among the original translation scores and scores (mean) obtained after perturbations.

scores_mean_map2 = np.mean(scores_map2[1:],axis=0) ## Computing means for the 20 perturbed scores.
scores_relative_diff_map2 = scores_map2[0] - scores_mean_map2 ## Computing relative difference among the original translation scores and scores (mean) obtained after perturbations.

scores_mean_map3 = np.mean(scores_map3[1:],axis=0) ## Computing means for the 20 perturbed scores.
scores_relative_diff_map3 = scores_map3[0] - scores_mean_map3 ## Computing relative difference among the original translation scores and scores (mean) obtained after perturbations.

scores_mean_map4 = np.mean(scores_map4[1:],axis=0) ## Computing means for the 20 perturbed scores.
scores_relative_diff_map4 = scores_map4[0] - scores_mean_map4 ## Computing relative difference among the original translation scores and scores (mean) obtained after perturbations.

scores_mean_map5 = np.mean(scores_map5[1:],axis=0) ## Computing means for the 20 perturbed scores.
scores_relative_diff_map5 = scores_map5[0] - scores_mean_map5 ## Computing relative difference among the original translation scores and scores (mean) obtained after perturbations.

scores_mean_map6_perts = np.mean(scores_map6_perts[0:],axis=0) ## Mean of 20 trials of perturbed scores.
scores_mean_map6_orig = np.mean(scores_map6_orig[0:],axis=0) ## Mean of 20 trails of original translation scores
scores_relative_diff_map6 = scores_mean_map6_orig - scores_mean_map6_perts ## Computing relative difference 

scores_mean_map7_perts = np.mean(scores_map7_perts[0:],axis=0) ## Mean of 20 trials of perturbed scores.
scores_mean_map7_orig = np.mean(scores_map7_orig[0:],axis=0) ## Mean of 20 trails of original translation scores
scores_relative_diff_map7 = scores_mean_map7_orig - scores_mean_map7_perts ## Computing relative difference 

scores_mean_map8 = np.mean(scores_map8[1:],axis=0) ## Computing means for the 20 perturbed scores.
scores_relative_diff_map8 = scores_map8[0] - scores_mean_map8 ## Computing relative difference among the original translation scores and scores (mean) obtained after perturbations.

In [ ]:
scores_mean_mpp1_perts = np.mean(scores_mpp1_perts[0:],axis=0) ## Mean of 20 trials of perturbed scores.
scores_mean_mpp1_orig = np.mean(scores_mpp1_orig[0:],axis=0) ## Mean of 20 trails of original translation scores
scores_relative_diff_mpp1 = scores_mean_mpp1_orig - scores_mean_mpp1_perts ## Computing relative difference among the original translation scores and scores (mean) obtained after perturbations.

scores_mean_mpp2_perts = np.mean(scores_mpp2_perts[0:],axis=0) ## Mean of 20 trials of perturbed scores.
scores_mean_mpp2_orig = np.mean(scores_mpp2_orig[0:],axis=0) ## Mean of 20 trails of original translation scores
scores_relative_diff_mpp2 = scores_mean_mpp2_orig - scores_mean_mpp2_perts ## Computing relative difference among the original translation scores and scores (mean) obtained after perturbations.

scores_mean_mpp3_perts = np.mean(scores_mpp3_perts[0:],axis=0) ## Mean of 20 trials of perturbed scores.
scores_mean_mpp3_orig = np.mean(scores_mpp3_orig[0:],axis=0) ## Mean of 20 trails of original translation scores
scores_relative_diff_mpp3 = scores_mean_mpp3_orig - scores_mean_mpp3_perts ## Computing relative difference among the original translation scores and scores (mean) obtained after perturbations.

scores_mean_mpp4_perts = np.mean(scores_mpp4_perts[0:],axis=0) ## Mean of 20 trials of perturbed scores.
scores_mean_mpp4_orig = np.mean(scores_mpp4_orig[0:],axis=0) ## Mean of 20 trails of original translation scores
scores_relative_diff_mpp4 = scores_mean_mpp4_orig - scores_mean_mpp4_perts ## Computing relative difference among the original translation scores and scores (mean) obtained after perturbations.

scores_mean_mpp5 = np.mean(scores_mpp5[1:],axis=0) ## Computing means for the 20 perturbed scores.
scores_relative_diff_mpp5 = scores_mpp5[0] - scores_mean_mpp5 ## Computing relative difference among the original translation scores and scores (mean) obtained after perturbations.

scores_mean_mpp6 = np.mean(scores_mpp6[1:],axis=0) ## Computing means for the 20 perturbed scores.
scores_relative_diff_mpp6 = scores_mpp6[0] - scores_mean_mpp6 ## Computing relative difference among the original translation scores and scores (mean) obtained after perturbations.

In [ ]:
map1_mean = np.mean(scores_relative_diff_map1)
map2_mean = np.mean(scores_relative_diff_map2)
map3_mean = np.mean(scores_relative_diff_map3)
map4_mean = np.mean(scores_relative_diff_map4)
map5_mean = np.mean(scores_relative_diff_map5)
map6_mean = np.mean(scores_relative_diff_map6)
map7_mean = np.mean(scores_relative_diff_map7)
map8_mean = np.mean(scores_relative_diff_map8)

mpp1_mean = np.mean(scores_relative_diff_mpp1)
mpp2_mean = np.mean(scores_relative_diff_mpp2)
mpp3_mean = np.mean(scores_relative_diff_mpp3)
mpp4_mean = np.mean(scores_relative_diff_mpp4)
mpp5_mean = np.mean(scores_relative_diff_mpp5)
mpp6_mean = np.mean(scores_relative_diff_mpp6)

In [ ]:
map1_var = np.var(scores_relative_diff_map1)
map2_var = np.var(scores_relative_diff_map2)
map3_var = np.var(scores_relative_diff_map3)
map4_var = np.var(scores_relative_diff_map4)
map5_var = np.var(scores_relative_diff_map5)
map6_var = np.var(scores_relative_diff_map6)
map7_var = np.var(scores_relative_diff_map7)
map8_var = np.var(scores_relative_diff_map8)

mpp1_var = np.var(scores_relative_diff_mpp1)
mpp2_var = np.var(scores_relative_diff_mpp2)
mpp3_var = np.var(scores_relative_diff_mpp3)
mpp4_var = np.var(scores_relative_diff_mpp4)
mpp5_var = np.var(scores_relative_diff_mpp5)
mpp6_var = np.var(scores_relative_diff_mpp6)

## Generating graphs and dumping scores

In [ ]:
X_mpp = np.array([mpp1_mean, mpp2_mean, mpp3_mean, mpp4_mean, mpp5_mean, mpp6_mean])
err_mpp = np.array([mpp1_var, mpp2_var, mpp3_var, mpp4_var, mpp5_var, mpp6_var])
rows = ["RemovePunctuation", "ReplacePunctuation", "RemoveDeterminers", "ReplaceDeterminers", "UPPERCASE", "lowercase"]
lang = []
for i in range(len(X_mpp)):
    lang.append(source_lang)

dfplot = pd.DataFrame(dict(Perturbations=rows, Differences=X_mpp, error=err_mpp, Language=lang))

fig = px.scatter(dfplot, title="Plotting Relative Difference in MPPs", y="Differences", x="Perturbations", error_y="error", color="Language")
fig.show()

In [ ]:
X_map = np.array([map1_mean, map2_mean, map3_mean, map4_mean, map5_mean, map6_mean, map7_mean, map8_mean])
err_map = np.array([map1_var, map2_var, map3_var, map4_var, map5_var, map6_var, map7_var, map8_var])
rows = ["RemoveNegation", "RemoveWords", "DuplicateWords", "InsertWords", "ReplaceWords", "BERTSentenceReplace", "ReplaceWordsWithAntonyms", "SourceAsTarget"]
lang = []
for i in range(len(X_map)):
    lang.append(source_lang)

dfplot = pd.DataFrame(dict(Perturbations=rows, Differences=X_map, error=err_map, Language=lang))

fig = px.scatter(dfplot, title="Plotting Relative Difference in MAPs", y="Differences", x="Perturbations", error_y="error", color="Language")
fig.show()

In [ ]:
X_both = np.array([mpp1_mean, mpp2_mean, mpp3_mean, mpp4_mean, mpp5_mean, mpp6_mean, map1_mean, map2_mean, map3_mean, map4_mean, map5_mean, map6_mean, map7_mean, map8_mean])
err_both = np.array([mpp1_var, mpp2_var, mpp3_var, mpp4_var, mpp5_var, mpp6_var, map1_var, map2_var, map3_var, map4_var, map5_var, map6_var, map7_var, map8_var])
rows = ["RemovePunctuation", "ReplacePunctuation", "RemoveDeterminers", "ReplaceDeterminers", "UPPERCASE", "lowercase", 
        "RemoveNegation", "RemoveWords", "DuplicateWords", "InsertWords", "ReplaceWords", "BERTSentenceReplace", "ReplaceWordsWithAntonyms", "SourceAsTarget"]
lang = ["MPP", "MPP", "MPP", "MPP", "MPP", "MPP", "MAP", "MAP", "MAP", "MAP", "MAP", "MAP", "MAP", "MAP"]
    
dfplot = pd.DataFrame(dict(Perturbations=rows, Differences=X_both, error=err_both, Language=lang))

fig = px.scatter(dfplot, title="Plotting Relative Difference in MPPs and MAPs", y="Differences", x="Perturbations", error_y="error", color="Language")
fig.show()

In [ ]:
!mkdir -p commonResults

In [ ]:
np.save("commonResults/"+source_lang+"Means_"+tqmodel, X_both)
np.save("commonResults/"+source_lang+"Errs_"+tqmodel, err_both)

## From here onwards, you have dumped the relative differences for all the perturbations in terms of mean values and variances. 
## You should do this for all five language pairs before being able to generate system rankings via "getPearsonandRanking.ipynb"